<a href="https://colab.research.google.com/github/esferita006/sql-en-python/blob/main/Base_cnsf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="Teal" face="Comic Sans MS,arial">
  <h1 align="center"><i>BASE DE DATOS</i></h1>
</font>

<font color="Black" face="Comic Sans MS,arial">
  <h5 align="center"><i>Acontinuacion se presentan una base de datos ficticia</i></h5>
</font>

In [ ]:
!pip install pyodbc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.7/334.7 kB 6.9 MB/s eta 0:00:00


In [ ]:

#Importar librerías
import pyodbc
import pandas as pd
import numpy as np
import os
import datetime

#Medición del tiempo de ejecución
startTime = datetime.datetime.now()


#Conexión a base
myconn=pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=CNSFSQLBI;DATABASE=RR8;Trusted_Connection=yes')

#Realizar directorios para importar y exportar archivos
#Directorio_imp=r'd:/Users/kluna/Documents/CNSF/2. Semáforo de Calidad de la Información Estadística (SCIE) Anual/4.16 RCV (Responsabilidad Civil)/1. Necesarios SCIE/2022/'
#Directorio_exp=r'd:/Users/kluna/Documents/CNSF/2. Semáforo de Calidad de la Información Estadística (SCIE) Anual/4.16 RCV (Responsabilidad Civil)/2. Repositorio/2022/'

Directorio_imp=r'd:/Users/SSDGDI01/Pictures/RC/IMP'
Directorio_exp=r'd:/Users/SSDGDI01/Pictures/RC/EXP'



#Importar las bases, es decir que compañías entregan y cuales compañías estan activas para tener una tabla base pivote
pivote=pd.read_csv(os.path.join(Directorio_imp,'Entrega por las compañías 9.csv'))
pivote['CLAVE_COMP']=pivote['CLAVE'].str.split('S',n=1, expand=True)[1]


#Variables definidas
año='2023' #Año de reporte que se quiere revisar
fecha_corte='2023-12-31' #Fecha de reporte que se quiere revisar / con formato AÑO-MES-DÍA
fecha_acumulada='2022-01-01'  #Fecha de inicio de vigencia para comparar PDA vs PE del año que se quiere revisar / con formato AÑO-MES-DÍA

factor_registros=2
factor_monto=4500
factor_otros=5

#-------------------------------------------------------------------------------------------------
#Revisión de compañías que entregan en 0's.

#Query para revisar quien entrego en 0's
entregas= pd.read_sql_query(f"""SELECT [CLAVE_COMP]
,COUNT(NUMPOL) AS TOTAL_POLIZAS
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}
GROUP BY CLAVE_COMP
ORDER BY CLAVE_COMP""", myconn)

entregas= pd.merge(left=pivote,right=entregas,left_on='CLAVE_COMP', right_on='CLAVE_COMP', how='left')
entregas['INFORMACION_CERO']=np.where(entregas['TOTAL_POLIZAS'].isnull(),'Si','No')
entregas.to_csv(os.path.join(Directorio_exp,'Entregas_Ceros.csv'),index=False,encoding='latin1')



#Datos Generales
dg = pd.read_sql_query(f"""
SELECT D.CLAVE_COMP,
    ROUND((SUM(CASE WHEN D.PRIMDEV = 0 AND YEAR(D.INIVIG) = D.ANIO_PARTICION THEN IIF(D.ESTATUS = '07', 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_1,
    ROUND((SUM(CASE WHEN D.ESTATUS IN ('07', '08') THEN IIF(D.FECHCANC IS NOT NULL, 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_2,
    ROUND((SUM(CASE WHEN D.FECHCANC IS NOT NULL THEN IIF(D.ESTATUS IN ('07', '08'), 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_3,
    ROUND((SUM(CASE WHEN D.FECHCANC IS NOT NULL THEN IIF(YEAR(D.FECHCANC) = D.ANIO_PARTICION, 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_4,
    ROUND((SUM(CASE WHEN D.FINVIG > D.FECHEMI THEN 0 ELSE 1 END) / CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_5,
    ROUND((SUM(CASE WHEN YEAR(D.FINVIG) >= D.ANIO_PARTICION THEN 0 ELSE 1 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_6,
    ROUND((SUM(CASE WHEN DATEDIFF(YEAR, D.INIVIG, D.FINVIG) <= 5 THEN 0 ELSE 1 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_7,
    ROUND((SUM(CASE WHEN D.VENTA IN ('06', '07') THEN IIF(D.COMDIR = 0, 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_8,
    ROUND((SUM(CASE WHEN D.TIPSEG != '11' THEN IIF(D.INIVIG < D.FINVIG, 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_9,
    ROUND((SUM(CASE WHEN D.PRIMDEV < 0 THEN IIF(D.PRIMEMI < 0, 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_10,
    ROUND((SUM(CASE WHEN D.PRIMDEV < 0 THEN IIF(D.ESTATUS = '07', 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_11,
    ROUND((SUM(CASE WHEN D.PRIMDEV = 0 AND D.ESTATUS = '07' THEN IIF(D.PRIMEMI <= 0, 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_12,
    ROUND((SUM(CASE WHEN D.ESTATUS IN ('01', '05') AND YEAR(D.FINVIG) = D.ANIO_PARTICION THEN IIF(D.PRIMDEV != 0, 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_13,
    ROUND((SUM(CASE WHEN D.ESTATUS = '07' AND D.MON = '10' THEN IIF(D.PRIMDEV = 0, 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_14,
    ROUND((SUM(CASE WHEN D.ESTATUS = '01' AND YEAR(D.FECHEMI) = D.ANIO_PARTICION THEN IIF(D.PRIMDEVACU < D.PRIMEMI, 0, 1) ELSE 0 END) / CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_15,
    ROUND((SUM(CASE WHEN D.ESTATUS = '05' THEN IIF(D.PRIMDEVACU >= D.PRIMEMI, 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_16,
    ROUND((SUM(CASE WHEN D.ESTATUS IN ('04', '08') AND YEAR(D.FECHEMI) = D.ANIO_PARTICION AND D.MON = '10' THEN IIF(D.PRIMDEVACU = D.PRIMEMI, 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_17,
    ROUND((SUM(CASE WHEN D.ESTATUS IN ('04', '08') AND YEAR(D.FECHEMI) < D.ANIO_PARTICION THEN IIF(D.PRIMDEVACU >= D.PRIMEMI, 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_18,
    ROUND((SUM(CASE WHEN D.ESTATUS IN ('03', '07') AND YEAR(D.FECHEMI) < D.ANIO_PARTICION THEN IIF(D.PRIMDEVACU >= D.PRIMEMI, 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_19,
    ROUND((SUM(CASE WHEN D.PRIMDEVACU = 0 AND D.INIVIG != '{fecha_corte}' THEN IIF(D.ESTATUS IN ('03', '07', '09'), 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_20,
    ROUND((SUM(CASE WHEN D.ESTATUS != '09' AND D.PRIMEMI > 0 AND D.INIVIG != '{fecha_corte}' THEN IIF(D.PRIMDEVACU > 0, 0, 1) ELSE 0 END) / CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_21,
    ROUND((SUM(CASE WHEN D.PRIMDEVACU < 0 THEN IIF((D.ESTATUS IN ('03', '07') AND D.MON = '20'), 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_22,
    ROUND((SUM(CASE WHEN D.ESTATUS = '07' AND D.MON = '10' THEN IIF(D.PRIMEMI <= 0, 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_23,
    ROUND((SUM(CASE WHEN D.ESTATUS != '09' AND D.PRIMDEVACU = 0 AND D.INIVIG != '{fecha_corte}' THEN IIF(D.PRIMEMI <= 0, 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_24,
    ROUND((SUM(CASE WHEN D.PRIMEMI > 0 AND YEAR(D.FECHEMI) = D.ANIO_PARTICION AND D.MON = '10' THEN IIF(D.ESTATUS != '07', 0, 1) ELSE 0 END) / CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_25,
    ROUND((SUM(CASE WHEN D.PRIMEMI = 0 THEN IIF((YEAR(D.FECHEMI) != D.ANIO_PARTICION OR D.ESTATUS = '07'), 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_26,
    ROUND((SUM(CASE WHEN D.PRIMEMI < 0 AND D.MON = '10' THEN IIF(YEAR(D.FECHEMI) != D.ANIO_PARTICION, 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_27,
    ROUND((SUM(CASE WHEN D.TIPSEG = '09' THEN IIF(D.GIRO <= '0023', 0, 1) ELSE 0 END) / CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS CON_VAL_28,
    ROUND((SUM(CASE WHEN D.ESTATUS IN ('01', '05') AND YEAR(D.INIVIG) < D.ANIO_PARTICION AND YEAR(D.FECHEMI) < D.ANIO_PARTICION THEN IIF(D.PRIMDEVACU > D.PRIMDEV, 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS CON_VAL_29,
    ROUND((SUM(CASE WHEN D.ESTATUS IN ('01', '05') AND YEAR(D.FECHEMI) = D.ANIO_PARTICION AND YEAR(D.INIVIG) = D.ANIO_PARTICION THEN IIF(D.PRIMDEVACU = D.PRIMDEV, 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS CON_VAL_30,
    ROUND((SUM(CASE WHEN D.ESTATUS IN ('04', '08') AND YEAR(D.INIVIG) < D.ANIO_PARTICION AND D.MON = '10' AND YEAR(D.FECHEMI) < D.ANIO_PARTICION THEN IIF(D.PRIMDEVACU > D.PRIMDEV, 0, 1) ELSE 0 END) / CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_31,
    ROUND((SUM(CASE WHEN D.ESTATUS IN ('04', '08') AND D.MON = '10' AND YEAR(D.FECHEMI) = D.ANIO_PARTICION AND YEAR(D.INIVIG) = D.ANIO_PARTICION THEN IIF(D.PRIMDEVACU = D.PRIMDEV, 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_32,
    ROUND((SUM(CASE WHEN D.ESTATUS IN ('03', '07') AND D.MON = '10' THEN IIF(D.PRIMDEVACU = D.PRIMDEV, 0, 1) ELSE 0 END)/ CAST(COUNT(CONCAT(D.NUMPOL, D.UBI, D.TIPSEG)) AS float)) * 100, 2) AS POR_VAL_33
FROM
(SELECT *
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION  = '2023') AS D
GROUP BY D.CLAVE_COMP
ORDER BY D.CLAVE_COMP
""", myconn)
dg = dg.add_suffix('_DG')


#Emisión
emi=pd.read_sql_query(f"""SELECT E.CLAVE_COMP
,ROUND((SUM(CASE WHEN D.INIVIG IS NOT NULL AND YEAR(D.INIVIG) <= E.ANIO_PARTICION AND YEAR(D.FINVIG) >= E.ANIO_PARTICION AND D.FECHCANC > D.INIVIG THEN IIF(E.SAEXP > 0, 0,1) ELSE 0 END)/CAST(COUNT(CONCAT(E.NUMPOL, E.UBI, E.TIPSEG)) as float))*100,2) AS PORC_VAL_1
FROM
(SELECT *
FROM [RR8].[dbo].[RR8_RC_EMI]
WHERE ANIO_PARTICION  =  {año}) E
INNER JOIN
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, UBI, TIPSEG, INIVIG, FINVIG, FECHCANC
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION  =  {año}) D
ON E.CLAVE_COMP=D.CLAVE_COMP AND E.NUMPOL = D.NUMPOL AND E.UBI = D.UBI AND E.TIPSEG = D.TIPSEG
GROUP BY E.CLAVE_COMP
ORDER BY E.CLAVE_COMP""", myconn)
emi = emi.add_suffix('_EMI')

#Siniestros
sin=pd.read_sql_query(f"""SELECT S.CLAVE_COMP
,ROUND((SUM(CASE WHEN D.INIVIG IS NULL THEN 0 WHEN S.FECHOCUR >= D.INIVIG THEN 0 ELSE 1 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_1
,ROUND((SUM(CASE WHEN D.FINVIG IS NULL THEN 0 WHEN S.FECHOCUR <= D.FINVIG THEN 0 ELSE 1 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_2
,ROUND((SUM(CASE WHEN D.FECHCANC IS NULL THEN 0 WHEN D.FECHCANC IS NOT NULL THEN IIF(S.FECHOCUR <= D.FECHCANC, 0,1) ELSE 0 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_3
,ROUND((SUM(CASE WHEN D.FECHEMI IS NULL THEN 0 WHEN S.FECHREPSIN >= D.FECHEMI THEN 0 ELSE 1 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_4
,ROUND((SUM(CASE WHEN S.MONTOCOAS > 0 THEN IIF(S.MONTOPAG > 0, 0,1) ELSE 0 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_5
,ROUND((SUM(CASE WHEN E.SA IS NOT NULL AND S.MONTOCOAS IS NOT NULL THEN IIF(S.MONTOCOAS < E.SA, 0,1) ELSE 0 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_6
,ROUND((SUM(CASE WHEN S.MONTODEDUC > 0 THEN IIF(S.MONTOPAG > 0, 0,1) ELSE 0 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_7
,ROUND((SUM(CASE WHEN E.SA IS NOT NULL AND S.MONTODEDUC IS NOT NULL THEN IIF(S.MONTODEDUC < E.SA, 0,1) ELSE 0 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_8
,ROUND((SUM(CASE WHEN D.MON IS NULL THEN 0 WHEN (S.ANIO_PARTICION = YEAR(S.FECHCONT) AND D.MON = '10') THEN IIF(S.MONTOSIN >= 0, 0,1) ELSE 0 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_9
,ROUND((SUM(CASE WHEN D.MON IS NULL THEN 0 WHEN (S.ANIO_PARTICION = YEAR(S.FECHCONT) AND D.MON = '10') THEN IIF(S.MONTOSIN >= S.SALV, 0,1) ELSE 0 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_10
,ROUND((SUM(CASE WHEN D.MON IS NULL THEN 0 WHEN (S.ANIO_PARTICION = YEAR(S.FECHCONT) AND D.MON = '10') THEN IIF(S.MONTOSIN >= S.MONTOPAG, 0,1) ELSE 0 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_11
,ROUND((SUM(CASE WHEN (S.ANIO_PARTICION = YEAR(FECHCONT) AND S.MONTOCOAS > 0) THEN IIF(S.MONTOSIN > S.MONTOCOAS, 0,1) ELSE 0 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_12
,ROUND((SUM(CASE WHEN E.SA IS NOT NULL THEN IIF(S.MONTOSIN <= E.SA, 0,1) ELSE 0 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_13
,ROUND((SUM(CASE WHEN D.MON IS NULL THEN 0 WHEN (S.ANIO_PARTICION = YEAR(S.FECHCONT) AND D.MON = '10') THEN IIF(S.MONTOSIN >= S.MONTORECUP, 0,1) ELSE 0 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_14
,ROUND((SUM(CASE WHEN S.MONTOPAG >= 0 THEN 0 ELSE 1 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_15
,ROUND((SUM(CASE WHEN E.SA IS NOT NULL THEN IIF(S.MONTOPAG <= E.SA, 0,1) ELSE 0 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_16
,ROUND((SUM(CASE WHEN D.MON IS NULL THEN 0 WHEN (YEAR(S.FECHCONT) = S.ANIO_PARTICION AND D.MON = '10') THEN IIF(S.MONTOSIN >= S.MONTORECUP, 0,1) ELSE 0 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_17
,ROUND((SUM(CASE WHEN D.MON IS NULL THEN 0 WHEN (YEAR(S.FECHCONT) = S.ANIO_PARTICION AND D.MON = '10') THEN IIF(S.MONTORECUP >= 0, 0,1) ELSE 0 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_18
,ROUND((SUM(CASE WHEN E.SA IS NULL THEN 0 WHEN E.SA > 0 THEN IIF(S.MONTORECUP <= E.SA, 0,1) ELSE 0 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_19
,ROUND((SUM(CASE WHEN D.MON IS NULL THEN 0 WHEN (YEAR(S.FECHCONT) = S.ANIO_PARTICION AND D.MON = '10') THEN IIF(S.MONTORECUP >= 0, 0,1) ELSE 0 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_20
,ROUND((SUM(CASE WHEN E.SA IS NULL THEN 0 WHEN S.SALV <= E.SA THEN 0 ELSE 1 END)/CAST(COUNT(CONCAT(S.NUMPOL, S.UBI, S.TIPSEG, S.ENTIDADSIN)) as float))*100,2) AS PORC_VAL_21
FROM
(SELECT *
FROM [RR8].[dbo].[RR8_RC_SIN]
WHERE ANIO_PARTICION = {año}) AS S
LEFT JOIN
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, TIPSEG, UBI, SA
FROM [RR8].[dbo].[RR8_RC_EMI]
WHERE ANIO_PARTICION = {año}) AS E
ON S.NUMPOL = E.NUMPOL AND S.UBI = E.UBI AND S.TIPSEG = E.TIPSEG
LEFT JOIN
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, TIPSEG, UBI, ENTIDAD, INIVIG, FINVIG, FECHCANC, FECHEMI, MON
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION = {año}) AS D
ON S.NUMPOL = D.NUMPOL AND S.UBI = D.UBI AND S.TIPSEG = D.TIPSEG
GROUP BY S.CLAVE_COMP
ORDER BY S.CLAVE_COMP
""", myconn)
sin = sin.add_suffix('_SIN')

#------------------------------RR8 VS RR7------------------------------------------------

#Datos generales

#Prima Emitida
prima_emi=pd.read_sql_query(f"""SELECT IIF(A.CLAVE_COMP IS NULL, B.CLAVE_COMP, A.CLAVE_COMP) AS CLAVE_COMP
,ABS(CASE WHEN ABS(ISNULL(B.PRIMA_EMI_RR7,0)-ISNULL(A.PRIMA_EMI_RR8,0))<4500 THEN 0
WHEN ABS(ISNULL(B.PRIMA_EMI_RR7,0)-ISNULL(A.PRIMA_EMI_RR8,0))>200000 THEN 100
WHEN PRIMA_EMI_RR8=0 AND PRIMA_EMI_RR7 IS NULL THEN 0
WHEN PRIMA_EMI_RR7=0 AND PRIMA_EMI_RR8 IS NULL THEN 0
WHEN (PRIMA_EMI_RR8=0 AND (PRIMA_EMI_RR7 IS NOT NULL OR PRIMA_EMI_RR7!=0)) THEN 100
WHEN (PRIMA_EMI_RR7=0 AND (PRIMA_EMI_RR8 IS NOT NULL OR PRIMA_EMI_RR8!=0)) THEN 100
WHEN (PRIMA_EMI_RR8 IS NULL AND (PRIMA_EMI_RR7 IS NOT NULL OR PRIMA_EMI_RR7!=0)) THEN 100
WHEN (PRIMA_EMI_RR7 IS NULL AND (PRIMA_EMI_RR8 IS NOT NULL OR PRIMA_EMI_RR8!=0)) THEN 100 ELSE
ABS(ROUND((1 - IIF(A.PRIMA_EMI_RR8 < B.PRIMA_EMI_RR7, A.PRIMA_EMI_RR8, B.PRIMA_EMI_RR7)
/IIF(A.PRIMA_EMI_RR8 > B.PRIMA_EMI_RR7, A.PRIMA_EMI_RR8, B.PRIMA_EMI_RR7)) * 100, 2)) END) AS '%DIF_PRIMEMI'
FROM (SELECT CLAVE_COMP,SUM(PRIMEMI) AS PRIMA_EMI_RR8
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}
GROUP BY CLAVE_COMP) A
LEFT JOIN (SELECT CLAVE_COMP, SUM(SALDO) AS PRIMA_EMI_RR7
FROM [RR7].[dbo].RR7_PRIMAS
WHERE CVE_RAMO = '040' AND CLAVE_PRIMAS = '040' AND FECHA_CORTE = '{fecha_corte}'
GROUP BY CLAVE_COMP) B
ON A.CLAVE_COMP = B.CLAVE_COMP""", myconn)

#Prima Retenida
prima_ret=pd.read_sql_query(f"""SELECT IIF(A.CLAVE_COMP IS NULL, B.CLAVE_COMP, A.CLAVE_COMP) AS CLAVE_COMP
,ABS((CASE WHEN ABS(ISNULL(B.PRIMA_RET_RR7,0)-ISNULL(A.PRIMA_RET_RR8,0))<4500 THEN 0
WHEN ABS(ISNULL(B.PRIMA_RET_RR7,0)-ISNULL(A.PRIMA_RET_RR8,0))>200000 THEN 100
WHEN PRIMA_RET_RR8=0 AND PRIMA_RET_RR7 IS NULL THEN 0
WHEN PRIMA_RET_RR7=0 AND PRIMA_RET_RR8 IS NULL THEN 0
WHEN (PRIMA_RET_RR8=0 AND (PRIMA_RET_RR7 IS NOT NULL OR PRIMA_RET_RR7!=0)) THEN 100
WHEN (PRIMA_RET_RR7=0 AND (PRIMA_RET_RR8 IS NOT NULL OR PRIMA_RET_RR8!=0)) THEN 100
WHEN (PRIMA_RET_RR8 IS NULL AND (PRIMA_RET_RR7 IS NOT NULL OR PRIMA_RET_RR7!=0)) THEN 100
WHEN (PRIMA_RET_RR7 IS NULL AND (PRIMA_RET_RR8 IS NOT NULL OR PRIMA_RET_RR8!=0)) THEN 100 ELSE
ABS(ROUND((1 - IIF(A.PRIMA_RET_RR8 < ISNULL(B.PRIMA_RET_RR7, 0), A.PRIMA_RET_RR8, ISNULL(B.PRIMA_RET_RR7, 0))
/IIF(A.PRIMA_RET_RR8 > ISNULL(B.PRIMA_RET_RR7, 0), A.PRIMA_RET_RR8, ISNULL(B.PRIMA_RET_RR7, 0))) * 100, 2)) END)) AS '%DIF_PRIMRET'
FROM
(SELECT CLAVE_COMP,SUM(PRIMRET) AS PRIMA_RET_RR8
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}
GROUP BY CLAVE_COMP) A
LEFT JOIN (SELECT PD.CLAVE_COMP,  (ISNULL(PD.PRIMA_DIR,0)-ISNULL(PC.PRIMA_CED,0)) AS PRIMA_RET_RR7
FROM
(SELECT CLAVE_COMP, SUM(SALDO) AS PRIMA_DIR
FROM [RR7].[dbo].RR7_PRIMAS
WHERE CVE_RAMO = '040' AND CLAVE_PRIMAS IN ('040') AND FECHA_CORTE = '{fecha_corte}'
GROUP BY CLAVE_COMP) PD
LEFT JOIN
(SELECT CLAVE_COMP, SUM(SALDO) AS PRIMA_CED
FROM [RR7].[dbo].RR7_PRIMAS
WHERE CVE_RAMO = '040' AND CLAVE_PRIMAS IN('180') AND FECHA_CORTE = '{fecha_corte}'
GROUP BY CLAVE_COMP) PC
ON PD.CLAVE_COMP=PC.CLAVE_COMP) B
ON A.CLAVE_COMP = B.CLAVE_COMP""", myconn)

#Siniestros

#Monto del Siniestro
monto_sin=pd.read_sql_query(f"""SELECT IIF(A.CLAVE_COMP IS NULL, B.CLAVE_COMP, A.CLAVE_COMP) AS CLAVE_COMP
,ABS((CASE WHEN ABS(ISNULL(B.MONTO_SIN_RR7,0)-ISNULL(A.MONTO_SIN_RR8,0))<4500 THEN 0
WHEN ABS(ISNULL(B.MONTO_SIN_RR7,0)-ISNULL(A.MONTO_SIN_RR8,0))>200000 THEN 100
WHEN MONTO_SIN_RR8=0 AND MONTO_SIN_RR7 IS NULL THEN 0
WHEN MONTO_SIN_RR7=0 AND MONTO_SIN_RR8 IS NULL THEN 0
WHEN (MONTO_SIN_RR8=0 AND (MONTO_SIN_RR7 IS NOT NULL OR MONTO_SIN_RR7!=0)) THEN 100
WHEN (MONTO_SIN_RR7=0 AND (MONTO_SIN_RR8 IS NOT NULL OR MONTO_SIN_RR8!=0)) THEN 100
WHEN (MONTO_SIN_RR8 IS NULL AND (MONTO_SIN_RR7 IS NOT NULL OR MONTO_SIN_RR7!=0)) THEN 100
WHEN (MONTO_SIN_RR7 IS NULL AND (MONTO_SIN_RR8 IS NOT NULL OR MONTO_SIN_RR8!=0)) THEN 100 ELSE
ABS(ROUND((1 - IIF(A.MONTO_SIN_RR8 < ISNULL(B.MONTO_SIN_RR7, 0), A.MONTO_SIN_RR8, ISNULL(B.MONTO_SIN_RR7, 0))
/IIF(A.MONTO_SIN_RR8 > ISNULL(B.MONTO_SIN_RR7, 0), A.MONTO_SIN_RR8, ISNULL(B.MONTO_SIN_RR7, 0)))*100,2)) END)) AS '%DIF_MONTOSIN'
FROM
(SELECT CLAVE_COMP, SUM(MONTOSIN) AS MONTO_SIN_RR8
FROM [RR8].[dbo].[RR8_RC_SIN]
WHERE ANIO_PARTICION = {año}
GROUP BY CLAVE_COMP) A
FULL JOIN (SELECT CLAVE_COMP, SUM(SALDO) as MONTO_SIN_RR7
FROM [RR7].[dbo].[RR7_COSTO_SINIESTRALIDAD]
WHERE (CLAVE_CTO_SIN = '050' AND CVE_RAMO = '040') AND FECHA_CORTE = '{fecha_corte}'
GROUP BY CLAVE_COMP) B
ON A.CLAVE_COMP = B.CLAVE_COMP
ORDER BY CLAVE_COMP""", myconn)

#Monto recuperado del reaseguro
monto_reas=pd.read_sql_query(f"""SELECT IIF(A.CLAVE_COMP IS NULL, B.CLAVE_COMP, A.CLAVE_COMP) AS CLAVE_COMP
,ABS((CASE WHEN ABS(ISNULL(B.MONTO_REAS_RR7,0)+ISNULL(A.MONTO_REAS_RR8,0))<4500 THEN 0
WHEN ABS(ISNULL(B.MONTO_REAS_RR7,0)+ISNULL(A.MONTO_REAS_RR8,0))>200000 THEN 100
WHEN MONTO_REAS_RR8=0 AND MONTO_REAS_RR7 IS NULL THEN 0
WHEN MONTO_REAS_RR7=0 AND MONTO_REAS_RR8 IS NULL THEN 0
WHEN (MONTO_REAS_RR8=0 AND (MONTO_REAS_RR7 IS NOT NULL OR MONTO_REAS_RR7!=0)) THEN 100
WHEN (MONTO_REAS_RR7=0 AND (MONTO_REAS_RR8 IS NOT NULL OR MONTO_REAS_RR8!=0)) THEN 100
WHEN (MONTO_REAS_RR8 IS NULL AND (MONTO_REAS_RR7 IS NOT NULL OR MONTO_REAS_RR7!=0)) THEN 100
WHEN (MONTO_REAS_RR7 IS NULL AND (MONTO_REAS_RR8 IS NOT NULL OR MONTO_REAS_RR8!=0)) THEN 100 ELSE
ABS(ROUND((1 + IIF(A.MONTO_REAS_RR8 < ISNULL(B.MONTO_REAS_RR7, 0), A.MONTO_REAS_RR8, ISNULL(B.MONTO_REAS_RR7, 0))
/IIF(A.MONTO_REAS_RR8 > ISNULL(B.MONTO_REAS_RR7, 0), A.MONTO_REAS_RR8, ISNULL(B.MONTO_REAS_RR7, 0)))*100,2)) END)) AS '%DIF_REAS'
FROM
(SELECT CLAVE_COMP, SUM(MONTOREAS) AS MONTO_REAS_RR8
FROM [RR8].[dbo].[RR8_RC_SIN]
WHERE ANIO_PARTICION = {año}
GROUP BY CLAVE_COMP) A
FULL JOIN (SELECT CLAVE_COMP, SUM(SALDO) as MONTO_REAS_RR7
FROM [RR7].[dbo].[RR7_COSTO_SINIESTRALIDAD]
WHERE (CLAVE_CTO_SIN = '130' AND CVE_RAMO = '040') AND FECHA_CORTE = '{fecha_corte}'
GROUP BY CLAVE_COMP) B
ON A.CLAVE_COMP = B.CLAVE_COMP
ORDER BY CLAVE_COMP""", myconn)

#Monto de recuperaciones de terceros
monto_recup=pd.read_sql_query(f"""SELECT IIF(A.CLAVE_COMP IS NULL, B.CLAVE_COMP, A.CLAVE_COMP) AS CLAVE_COMP
,ABS((CASE WHEN ABS(ISNULL(B.MONTO_RECUP_RR7,0)+ISNULL(A.MONTO_RECUP_RR8,0))<4500 THEN 0
WHEN ABS(ISNULL(B.MONTO_RECUP_RR7,0)+ISNULL(A.MONTO_RECUP_RR8,0))>200000 THEN 100
WHEN MONTO_RECUP_RR8=0 AND MONTO_RECUP_RR7 IS NULL THEN 0
WHEN MONTO_RECUP_RR7=0 AND MONTO_RECUP_RR8 IS NULL THEN 0
WHEN (MONTO_RECUP_RR8=0 AND (MONTO_RECUP_RR7 IS NOT NULL OR MONTO_RECUP_RR7!=0)) THEN 100
WHEN (MONTO_RECUP_RR7=0 AND (MONTO_RECUP_RR8 IS NOT NULL OR MONTO_RECUP_RR8!=0)) THEN 100
WHEN (MONTO_RECUP_RR8 IS NULL AND (MONTO_RECUP_RR7 IS NOT NULL OR MONTO_RECUP_RR7!=0)) THEN 100
WHEN (MONTO_RECUP_RR7 IS NULL AND (MONTO_RECUP_RR8 IS NOT NULL OR MONTO_RECUP_RR8!=0)) THEN 100 ELSE
ABS(ROUND((1 + IIF(A.MONTO_RECUP_RR8 < ISNULL(B.MONTO_RECUP_RR7, 0), A.MONTO_RECUP_RR8, ISNULL(B.MONTO_RECUP_RR7, 0))
/IIF(A.MONTO_RECUP_RR8 > ISNULL(B.MONTO_RECUP_RR7, 0), A.MONTO_RECUP_RR8, ISNULL(B.MONTO_RECUP_RR7, 0)))*100,2)) END)) AS '%DIF_MONTORECUP'
FROM
(SELECT CLAVE_COMP, SUM(MONTORECUP) AS MONTO_RECUP_RR8
FROM [RR8].[dbo].[RR8_RC_SIN]
WHERE ANIO_PARTICION = {año}
GROUP BY CLAVE_COMP) A
FULL JOIN (SELECT CLAVE_COMP, SUM(SALDO) as MONTO_RECUP_RR7
FROM [RR7].[dbo].[RR7_COSTO_SINIESTRALIDAD]
WHERE (CLAVE_CTO_SIN = '200' AND CVE_RAMO = '040') AND FECHA_CORTE = '{fecha_corte}'
GROUP BY CLAVE_COMP) B
ON A.CLAVE_COMP = B.CLAVE_COMP
ORDER BY CLAVE_COMP""", myconn)

#Monto de Salvamento

monto_salv=pd.read_sql_query(f"""SELECT IIF(A.CLAVE_COMP IS NULL, B.CLAVE_COMP, A.CLAVE_COMP) AS CLAVE_COMP
,ABS((CASE WHEN ABS(ISNULL(B.MONTO_SALV_RR7,0)+ISNULL(A.MONTO_SALV_RR8,0))<4500 THEN 0
WHEN ABS(ISNULL(B.MONTO_SALV_RR7,0)+ISNULL(A.MONTO_SALV_RR8,0))>200000 THEN 100
WHEN MONTO_SALV_RR8=0 AND MONTO_SALV_RR7 IS NULL THEN 0
WHEN MONTO_SALV_RR7=0 AND MONTO_SALV_RR8 IS NULL THEN 0
WHEN (MONTO_SALV_RR8=0 AND (MONTO_SALV_RR7 IS NOT NULL OR MONTO_SALV_RR7!=0)) THEN 100
WHEN (MONTO_SALV_RR7=0 AND (MONTO_SALV_RR8 IS NOT NULL OR MONTO_SALV_RR8!=0)) THEN 100
WHEN (MONTO_SALV_RR8 IS NULL AND (MONTO_SALV_RR7 IS NOT NULL OR MONTO_SALV_RR7!=0)) THEN 100
WHEN (MONTO_SALV_RR7 IS NULL AND (MONTO_SALV_RR8 IS NOT NULL OR MONTO_SALV_RR8!=0)) THEN 100 ELSE
ABS(ROUND((1 + IIF(A.MONTO_SALV_RR8 < ISNULL(B.MONTO_SALV_RR7, 0), A.MONTO_SALV_RR8, ISNULL(B.MONTO_SALV_RR7, 0))
/IIF(A.MONTO_SALV_RR8 > ISNULL(B.MONTO_SALV_RR7, 0), A.MONTO_SALV_RR8, ISNULL(B.MONTO_SALV_RR7, 0)))*100,2)) END)) AS '%DIF_SALV'
FROM
(SELECT CLAVE_COMP, SUM(SALV) AS MONTO_SALV_RR8
FROM [RR8].[dbo].[RR8_RC_SIN]
WHERE ANIO_PARTICION = {año}
GROUP BY CLAVE_COMP) A
FULL JOIN (SELECT CLAVE_COMP, SUM(SALDO) as MONTO_SALV_RR7
FROM [RR7].[dbo].[RR7_COSTO_SINIESTRALIDAD]
WHERE (CLAVE_CTO_SIN = '190' AND CVE_RAMO = '040') AND FECHA_CORTE = '{fecha_corte}'
GROUP BY CLAVE_COMP) B
ON A.CLAVE_COMP = B.CLAVE_COMP
ORDER BY CLAVE_COMP""", myconn)

contables = (prima_emi.merge(prima_ret, on=['CLAVE_COMP'], how='left')
         .merge(monto_sin, on=['CLAVE_COMP'], how='left')
         .merge(monto_reas, on=['CLAVE_COMP'], how='left')
         .merge(monto_recup, on=['CLAVE_COMP'], how='left')
         .merge(monto_salv, on=['CLAVE_COMP'], how='left'))

########## Pólizas con un 5% mayor en la etiqueta de otros ##########

#Datos generales

otros_venta=pd.read_sql_query(f"""SELECT P.CLAVE_COMP
,ROUND((O.OTRA_VENTA/CAST(P.NUM_POL AS FLOAT))*100,2) AS '%OTROS_VENTA'
FROM
(SELECT CLAVE_COMP, COUNT(NUMPOL) AS NUM_POL
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}
GROUP BY CLAVE_COMP) P
LEFT JOIN
(SELECT CLAVE_COMP, COUNT(NUMPOL) AS OTRA_VENTA
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año} AND VENTA='99'
GROUP BY CLAVE_COMP) O
ON P.CLAVE_COMP=O.CLAVE_COMP
ORDER BY P.CLAVE_COMP""", myconn)

otros_seg=pd.read_sql_query(f"""SELECT P.CLAVE_COMP
,ROUND((O.OTRO_TIPSEG_DG/CAST(P.NUM_POL AS FLOAT))*100,2) AS '%OTROS_TIPSEG_DG'
FROM
(SELECT CLAVE_COMP, COUNT(NUMPOL) AS NUM_POL
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}
GROUP BY CLAVE_COMP) P
LEFT JOIN
(SELECT CLAVE_COMP, COUNT(NUMPOL) AS OTRO_TIPSEG_DG
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año} AND TIPSEG='99'
GROUP BY CLAVE_COMP) O
ON P.CLAVE_COMP=O.CLAVE_COMP
ORDER BY P.CLAVE_COMP""", myconn)

otros_estatus=pd.read_sql_query(f"""SELECT P.CLAVE_COMP
,ROUND((O.OTRA_ESTATUS/CAST(P.NUM_POL AS FLOAT))*100,2) AS '%OTROS_ESTATUS'
FROM
(SELECT CLAVE_COMP, COUNT(NUMPOL) AS NUM_POL
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}
GROUP BY CLAVE_COMP) P
LEFT JOIN
(SELECT CLAVE_COMP, COUNT(NUMPOL) AS OTRA_ESTATUS
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año} AND ESTATUS='99'
GROUP BY CLAVE_COMP) O
ON P.CLAVE_COMP=O.CLAVE_COMP
ORDER BY P.CLAVE_COMP""", myconn)

otros_cotizacion=pd.read_sql_query(f"""SELECT P.CLAVE_COMP
,ROUND((O.OTRA_BASE/CAST(P.NUM_POL AS FLOAT))*100,2) AS '%OTROS_BASCOT'
FROM
(SELECT CLAVE_COMP, COUNT(NUMPOL) AS NUM_POL
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}
GROUP BY CLAVE_COMP) P
LEFT JOIN
(SELECT CLAVE_COMP, COUNT(NUMPOL) AS OTRA_BASE
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año} AND BASCOT='99'
GROUP BY CLAVE_COMP) O
ON P.CLAVE_COMP=O.CLAVE_COMP
ORDER BY P.CLAVE_COMP""", myconn)

#Emision

otros_cob_emi=pd.read_sql_query(f"""SELECT P.CLAVE_COMP
,ROUND((O.OTRO_COB/CAST(P.NUM_POL AS FLOAT))*100,2) AS '%OTROS_COB_EMI'
FROM
(SELECT CLAVE_COMP, COUNT(NUMPOL) AS NUM_POL
FROM [RR8].[dbo].[RR8_RC_EMI]
WHERE ANIO_PARTICION={año}
GROUP BY CLAVE_COMP) P
LEFT JOIN
(SELECT CLAVE_COMP, COUNT(NUMPOL) AS OTRO_COB
FROM [RR8].[dbo].[RR8_RC_EMI]
WHERE ANIO_PARTICION={año} AND COB='99'
GROUP BY CLAVE_COMP) O
ON P.CLAVE_COMP=O.CLAVE_COMP
ORDER BY P.CLAVE_COMP""", myconn)

otros_seg_emi=pd.read_sql_query(f"""SELECT P.CLAVE_COMP
,ROUND((O.OTRO_TIPSEG_EMI/CAST(P.NUM_POL AS FLOAT))*100,2) AS '%OTROS_TIPSEG_EMI'
FROM
(SELECT CLAVE_COMP, COUNT(NUMPOL) AS NUM_POL
FROM [RR8].[dbo].[RR8_RC_EMI]
WHERE ANIO_PARTICION={año}
GROUP BY CLAVE_COMP) P
LEFT JOIN
(SELECT CLAVE_COMP, COUNT(NUMPOL) AS OTRO_TIPSEG_EMI
FROM [RR8].[dbo].[RR8_RC_EMI]
WHERE ANIO_PARTICION={año} AND TIPSEG='99'
GROUP BY CLAVE_COMP) O
ON P.CLAVE_COMP=O.CLAVE_COMP
ORDER BY P.CLAVE_COMP""", myconn)

#Siniestros
otros_seg_sin=pd.read_sql_query(f"""SELECT P.CLAVE_COMP
,ROUND((O.OTRO_TIPSEG_SIN/CAST(P.NUM_POL AS FLOAT))*100,2) AS '%OTROS_TIPSEG_SIN'
FROM
(SELECT CLAVE_COMP, COUNT(NUMPOL) AS NUM_POL
FROM [RR8].[dbo].[RR8_RC_SIN]
WHERE ANIO_PARTICION={año}
GROUP BY CLAVE_COMP) P
LEFT JOIN
(SELECT CLAVE_COMP, COUNT(NUMPOL) AS OTRO_TIPSEG_SIN
FROM [RR8].[dbo].[RR8_RC_SIN]
WHERE ANIO_PARTICION={año} AND TIPSEG='99'
GROUP BY CLAVE_COMP) O
ON P.CLAVE_COMP=O.CLAVE_COMP
ORDER BY P.CLAVE_COMP""", myconn)

otros_cob_sin=pd.read_sql_query(f"""SELECT P.CLAVE_COMP
,ROUND((O.OTRO_COB/CAST(P.NUM_POL AS FLOAT))*100,2) AS '%OTROS_COB_SIN'
FROM
(SELECT CLAVE_COMP, COUNT(NUMPOL) AS NUM_POL
FROM [RR8].[dbo].[RR8_RC_SIN]
WHERE ANIO_PARTICION={año}
GROUP BY CLAVE_COMP) P
LEFT JOIN
(SELECT CLAVE_COMP, COUNT(NUMPOL) AS OTRO_COB
FROM [RR8].[dbo].[RR8_RC_SIN]
WHERE ANIO_PARTICION={año} AND COB='99'
GROUP BY CLAVE_COMP) O
ON P.CLAVE_COMP=O.CLAVE_COMP
ORDER BY P.CLAVE_COMP""", myconn)

otros = (otros_venta.merge(otros_seg, on=['CLAVE_COMP'], how='left')
         .merge(otros_estatus, on=['CLAVE_COMP'], how='left')
         .merge(otros_cotizacion, on=['CLAVE_COMP'], how='left')
         .merge(otros_cob_emi, on=['CLAVE_COMP'], how='left')
         .merge(otros_seg_emi, on=['CLAVE_COMP'], how='left')
         .merge(otros_seg_sin, on=['CLAVE_COMP'], how='left')
         .merge(otros_cob_sin, on=['CLAVE_COMP'], how='left')
         )

#-----------------------------------------------Consistencia de catálogo entre años-----------------------

#---Datos generales---

#Moneda
mon=pd.read_sql_query(f"""SELECT A.CLAVE_COMP
,ROUND((SUM(IIF(A.MON = B.MON, 0, 1))/CAST(COUNT(A.NUMPOL) AS FLOAT))*100,2) AS '%DIF_MON'
FROM
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, UBI, TIPSEG, MON
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}) A
JOIN
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, UBI, TIPSEG, MON
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}-1) B
ON A.CLAVE_COMP = B.CLAVE_COMP AND A.NUMPOL = B.NUMPOL AND A.UBI = B.UBI AND A.TIPSEG = B.TIPSEG
GROUP BY A.CLAVE_COMP
ORDER BY A.CLAVE_COMP""", myconn)

#Forma de venta
venta=pd.read_sql_query(f"""SELECT A.CLAVE_COMP
,ROUND((SUM(IIF(A.VENTA = B.VENTA, 0, 1))/CAST(COUNT(A.NUMPOL) AS FLOAT))*100,2) AS '%DIF_FVENTA'
FROM
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, UBI, TIPSEG, VENTA
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}) A
JOIN
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, UBI, TIPSEG, VENTA
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}-1) B
ON A.CLAVE_COMP = B.CLAVE_COMP AND A.NUMPOL = B.NUMPOL AND A.UBI = B.UBI AND A.TIPSEG = B.TIPSEG
GROUP BY A.CLAVE_COMP
ORDER BY A.CLAVE_COMP""", myconn)

#Giro
giro=pd.read_sql_query(f"""SELECT A.CLAVE_COMP
,ROUND((SUM(IIF(A.GIRO = B.GIRO, 0, 1))/CAST(COUNT(A.NUMPOL) AS FLOAT))*100,2) AS '%DIF_GIRO'
FROM
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, UBI, TIPSEG, GIRO
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}) A
JOIN
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, UBI, TIPSEG, GIRO
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}-1) B
ON A.CLAVE_COMP = B.CLAVE_COMP AND A.NUMPOL = B.NUMPOL AND A.UBI = B.UBI AND A.TIPSEG = B.TIPSEG
GROUP BY A.CLAVE_COMP
ORDER BY A.CLAVE_COMP""",myconn)

#Entidad DG
entidad_dg=pd.read_sql_query(f"""SELECT A.CLAVE_COMP
,ROUND((SUM(IIF(A.ENTIDAD = B.ENTIDAD, 0, 1))/CAST(COUNT(A.NUMPOL) AS FLOAT))*100,2) AS '%DIF_ENT_DG'
FROM
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, UBI, TIPSEG, ENTIDAD
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}) A
JOIN
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, UBI, TIPSEG, ENTIDAD
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}-1) B
ON A.CLAVE_COMP = B.CLAVE_COMP AND A.NUMPOL = B.NUMPOL AND A.UBI = B.UBI AND A.TIPSEG = B.TIPSEG
GROUP BY A.CLAVE_COMP
ORDER BY A.CLAVE_COMP""",myconn)

#Esquema de aseguramiento
esq_aseg=pd.read_sql_query(f"""SELECT A.CLAVE_COMP
,ROUND((SUM(IIF(A.ESQASEG = B.ESQASEG, 0, 1))/CAST(COUNT(A.NUMPOL) AS FLOAT))*100,2) AS '%DIF_ESQ_ASEG'
FROM
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, UBI,TIPSEG, ESQASEG
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}) A
JOIN
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, UBI,TIPSEG , ESQASEG
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}-1) B
ON A.CLAVE_COMP = B.CLAVE_COMP AND A.NUMPOL = B.NUMPOL AND A.UBI = B.UBI AND A.TIPSEG = B.TIPSEG
GROUP BY A.CLAVE_COMP
ORDER BY A.CLAVE_COMP""", myconn)

#Subtipo de seguro
subtip=pd.read_sql_query(f"""SELECT A.CLAVE_COMP
,ROUND((SUM(IIF(A.SUBTIP = B.SUBTIP, 0, 1))/CAST(COUNT(A.NUMPOL) AS FLOAT))*100,2) AS '%DIF_SUBSEG'
FROM
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, UBI, TIPSEG, SUBTIP
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}) A
JOIN
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, UBI, TIPSEG, SUBTIP
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}-1) B
ON A.CLAVE_COMP = B.CLAVE_COMP AND A.NUMPOL = B.NUMPOL AND A.UBI = B.UBI AND A.TIPSEG = B.TIPSEG
GROUP BY A.CLAVE_COMP
ORDER BY A.CLAVE_COMP""", myconn)

#Base de cotizacion
base_cot=pd.read_sql_query(f"""SELECT A.CLAVE_COMP
,ROUND((SUM(IIF(A.BASCOT = B.BASCOT, 0, 1))/CAST(COUNT(A.NUMPOL) AS FLOAT))*100,2) AS '%DIF_BASCOT'
FROM
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, UBI, TIPSEG, BASCOT
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}) A
JOIN
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, UBI, TIPSEG, BASCOT
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}-1) B
ON A.CLAVE_COMP = B.CLAVE_COMP AND A.NUMPOL = B.NUMPOL AND A.UBI = B.UBI AND A.TIPSEG = B.TIPSEG
GROUP BY A.CLAVE_COMP
ORDER BY A.CLAVE_COMP""", myconn)

#Entidad siniestros
entidad_sin=pd.read_sql_query(f"""SELECT A.CLAVE_COMP
,ROUND((SUM(IIF(A.ENTIDADSIN = B.ENTIDADSIN, 0, 1))/CAST(COUNT(A.NUMPOL) AS FLOAT))*100,2) AS '%DIF_ENT_SIN'
FROM
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, UBI, TIPSEG, ENTIDADSIN, COB, NUMSIN
FROM [RR8].[dbo].[RR8_RC_SIN]
WHERE ANIO_PARTICION={año}) A
JOIN
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, UBI, TIPSEG, ENTIDADSIN, COB, NUMSIN
FROM [RR8].[dbo].[RR8_RC_SIN]
WHERE ANIO_PARTICION={año}-1) B
ON A.CLAVE_COMP = B.CLAVE_COMP AND A.NUMPOL = B.NUMPOL AND A.UBI = B.UBI AND A.TIPSEG = B.TIPSEG AND A.COB = B.COB AND A.NUMSIN=B.NUMSIN
GROUP BY A.CLAVE_COMP
ORDER BY A.CLAVE_COMP""",myconn)

#Causa siniestro
causin=pd.read_sql_query(f"""SELECT A.CLAVE_COMP
,ROUND((SUM(IIF(A.CAUSIN = B.CAUSIN OR B.CAUSIN IS NULL, 0, 1))/CAST(COUNT(A.NUMPOL) AS FLOAT))*100,2) AS '%DIF_CAUSIN'
FROM
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, UBI, TIPSEG, CAUSIN, COB, NUMSIN
FROM [RR8].[dbo].[RR8_RC_SIN]
WHERE ANIO_PARTICION={año}) A
LEFT JOIN
(SELECT CLAVE_COMP, NUMPOL, ANIO_PARTICION, UBI, TIPSEG, CAUSIN, COB, NUMSIN
FROM [RR8].[dbo].[RR8_RC_SIN]
WHERE ANIO_PARTICION={año}-1) B
ON A.CLAVE_COMP = B.CLAVE_COMP AND A.NUMPOL = B.NUMPOL AND A.UBI = B.UBI AND A.TIPSEG = B.TIPSEG AND A.COB = B.COB AND A.NUMSIN=B.NUMSIN
GROUP BY A.CLAVE_COMP
ORDER BY A.CLAVE_COMP""",myconn)

cons = (mon.merge(venta, on=['CLAVE_COMP'], how='left')
         .merge(giro, on=['CLAVE_COMP'], how='left')
         .merge(entidad_dg, on=['CLAVE_COMP'], how='left')
         .merge(esq_aseg, on=['CLAVE_COMP'], how='left')
         .merge(subtip, on=['CLAVE_COMP'], how='left')
         .merge(base_cot, on=['CLAVE_COMP'], how='left')
         .merge(entidad_sin, on=['CLAVE_COMP'], how='left')
         .merge(causin, on=['CLAVE_COMP'], how='left')
         )




#-----------------------------------------------Años anteriores-----------------------

aant_acum=pd.read_sql_query(f"""SELECT A.CLAVE_COMP
,ROUND(ABS(1-SUM(A.PRIMDEVACU)/SUM(COALESCE(A.PRIMEMI,0)+COALESCE(A21.PRIMEMI,0)+COALESCE(A20.PRIMEMI,0)))*100,2) AS '%DIF_PDA'
FROM
(SELECT * FROM [RR8].[dbo].[RR8_RC_DG] WHERE ANIO_PARTICION={año} AND ESTATUS='05' AND INIVIG>='{fecha_acumulada}')A
LEFT JOIN
(SELECT * FROM [RR8].[dbo].[RR8_RC_DG] WHERE ANIO_PARTICION={año}-1)A21
ON A.CLAVE_COMP=A21.CLAVE_COMP AND A.NUMPOL=A21.NUMPOL AND A.UBI=A21.UBI AND A.TIPSEG = A21.TIPSEG
LEFT JOIN
(SELECT * FROM [RR8].[dbo].[RR8_RC_DG] WHERE ANIO_PARTICION={año}-2)A20
ON A.CLAVE_COMP=A20.CLAVE_COMP AND A.NUMPOL=A20.NUMPOL AND A.UBI=A20.UBI AND A.TIPSEG = A20.TIPSEG
GROUP BY A.CLAVE_COMP""", myconn)

aant_vigor=pd.read_sql_query(f"""SELECT A.CLAVE_COMP
,ROUND((1-COUNT(B.NUMPOL)/CAST(COUNT(A.NUMPOL) AS FLOAT))*100,2) AS '%VIGFALTANTES'
FROM
(SELECT CLAVE_COMP, ANIO_PARTICION, NUMPOL, UBI, TIPSEG
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}-1 AND ESTATUS='01') A
LEFT JOIN
(SELECT CLAVE_COMP, ANIO_PARTICION, NUMPOL, UBI, TIPSEG
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}) B
ON A.CLAVE_COMP=B.CLAVE_COMP AND A.NUMPOL=B.NUMPOL AND A.UBI=B.UBI AND A.TIPSEG = B.TIPSEG
GROUP BY A.CLAVE_COMP
ORDER BY A.CLAVE_COMP""", myconn)

aant_sin=pd.read_sql_query(f"""SELECT A.CLAVE_COMP
,ROUND((SUM(IIF(B.CLAVE_COMP IS NULL,1,0))/CAST(COUNT(A.NUMPOL) AS float))*100, 2) AS '%FALTANTES'
FROM
(SELECT CLAVE_COMP, NUMPOL, UBI, TIPSEG, COB, NUMSIN
FROM [RR8].[dbo].[RR8_RC_SIN]
WHERE ANIO_PARTICION={año} AND YEAR(FECHCONT)={año}-1) A
LEFT JOIN
(SELECT CLAVE_COMP, NUMPOL, UBI, TIPSEG, COB, NUMSIN
FROM [RR8].[dbo].[RR8_RC_SIN]
WHERE ANIO_PARTICION={año}-1) B
ON A.CLAVE_COMP = B.CLAVE_COMP AND A.NUMPOL = B.NUMPOL AND A.UBI = B.UBI AND A.TIPSEG = B.TIPSEG AND A.COB = B.COB AND A.NUMSIN=B.NUMSIN
GROUP BY A.CLAVE_COMP
ORDER BY A.CLAVE_COMP""", myconn)

aant_sinrep=pd.read_sql_query(f"""SELECT A.CLAVE_COMP
,ROUND((SUM(IIF(B.CLAVE_COMP IS NULL,1,0))/CAST(COUNT(A.NUMPOL) AS float))*100, 2) AS '%SINREP'
FROM
(SELECT CLAVE_COMP, NUMPOL, UBI, TIPSEG, COB
FROM [RR8].[dbo].[RR8_RC_SIN]
WHERE ANIO_PARTICION={año} AND YEAR(FECHOCUR)={año}) A
LEFT JOIN
(SELECT CLAVE_COMP, NUMPOL, UBI, TIPSEG, COB
FROM [RR8].[dbo].[RR8_RC_EMI]
WHERE ANIO_PARTICION={año}) B
ON A.CLAVE_COMP=B.CLAVE_COMP AND A.NUMPOL=B.NUMPOL AND A.UBI=B.UBI AND A.TIPSEG = B.TIPSEG AND A.COB = B.COB
GROUP BY A.CLAVE_COMP
ORDER BY A.CLAVE_COMP""",myconn)

reg_dgemi=pd.read_sql_query(f"""SELECT D.CLAVE_COMP
,ROUND(SUM(IIF(E.NUMPOL IS NULL, 1, 0))/CAST(COUNT(DISTINCT(CONCAT(D.NUMPOL,D.UBI,D.TIPSEG))) AS float) * 100, 2) AS '%DIF_REG_DGEMI'
FROM
(SELECT *
FROM [RR8].[DBO].[RR8_RC_DG]
WHERE ANIO_PARTICION={año}) D
LEFT JOIN
(SELECT *
FROM
[RR8].[DBO].[RR8_RC_EMI]
WHERE ANIO_PARTICION={año}) E
ON D.CLAVE_COMP = E.CLAVE_COMP AND D.NUMPOL = E.NUMPOL AND D.UBI = E.UBI AND D.TIPSEG = E.TIPSEG
WHERE YEAR(INIVIG) <= D.ANIO_PARTICION AND YEAR(FINVIG) >= D.ANIO_PARTICION
GROUP BY D.CLAVE_COMP""",myconn)


#/*Si estatus = (03, 04, 07, 08) entonces devacum = Σ primemi (tres ultimos años)*/
est=pd.read_sql_query(f"""SELECT CLAVE_COMP
, ROUND(ABS(SUM(RESULTADO)) * 100.0 / IIF(SUM(PRIMEMI_TOTAL) = 0, 1, SUM(PRIMEMI_TOTAL)), 2) AS ERROR_PORC_EST1
FROM ( SELECT D.CLAVE_COMP,D.NUMPOL,D.MON,SUM(D.PRIMDEVACU) AS SUM_PRIM
,SUM(COALESCE(D.PRIMEMI, 0)) AS PE_AÑO_ACTUAL
,SUM(COALESCE(E.PRIMEMI, 0)) AS PE_AÑO_PENULTIMO
,SUM(COALESCE(F.PRIMEMI, 0)) AS PE_AÑO_ULTIMO
,SUM(D.PRIMDEVACU) AS PRIM_D,
IIF(SUM(COALESCE(D.PRIMEMI, 0)) + SUM(COALESCE(E.PRIMEMI, 0)) + SUM(COALESCE(F.PRIMEMI, 0)) = SUM(D.PRIMDEVACU),0
,SUM(COALESCE(D.PRIMEMI, 0)) + SUM(COALESCE(E.PRIMEMI, 0)) + SUM(COALESCE(F.PRIMEMI, 0)) - SUM(D.PRIMDEVACU)) AS RESULTADO,
SUM(COALESCE(D.PRIMEMI, 0)) + SUM(COALESCE(E.PRIMEMI, 0)) + SUM(COALESCE(F.PRIMEMI, 0)) AS PRIMEMI_TOTAL
FROM [RR8].[dbo].[RR8_RC_DG]  D
    LEFT JOIN [RR8].[dbo].[RR8_RC_DG]  E
    ON D.CLAVE_COMP = E.CLAVE_COMP AND D.NUMPOL = E.NUMPOL AND E.ANIO_PARTICION = {año}-1
    LEFT JOIN[RR8].[dbo].[RR8_RC_DG]  F
    ON D.CLAVE_COMP = F.CLAVE_COMP AND D.NUMPOL = F.NUMPOL AND F.ANIO_PARTICION = {año}-2
    WHERE D.ANIO_PARTICION ={año} AND D.INIVIG >= '{fecha_acumulada}' AND D.ESTATUS IN ('03', '04', '07', '08') AND (D.MON!='20' AND D.FINVIG <='{fecha_corte}' )
GROUP BY D.CLAVE_COMP,D.NUMPOL,D.MON
) J
GROUP BY CLAVE_COMP""", myconn)

#/*Si estatus = 01 entonces devacum < Σ primemi (tres ultimos años)*/
est_1=pd.read_sql_query(f"""SELECT CLAVE_COMP,
CASE
    WHEN SUM(PRIMEMI_TOTAL) = 0 THEN 0
    ELSE ROUND(ABS(SUM(RESULTADO) / SUM(PRIMEMI_TOTAL)) * 100, 5)  END AS ERROR_PORC_EST2
FROM (
SELECT D.CLAVE_COMP,D.NUMPOL,D.PRIMEMI AS ACTUAL,E.PRIMEMI AS ANTERIOR,F.PRIMEMI AS ANTEANTERIOR, D.PRIMDEVACU AS PDA,
       IIF( (D.PRIMDEVACU) < (COALESCE((D.PRIMEMI), 0) + COALESCE((E.PRIMEMI), 0) + COALESCE((F.PRIMEMI), 0)),0,
           (COALESCE((D.PRIMEMI), 0) + COALESCE((E.PRIMEMI), 0) + COALESCE((F.PRIMEMI), 0)) - (D.PRIMDEVACU)) AS RESULTADO,
       (COALESCE((D.PRIMEMI), 0) + COALESCE((E.PRIMEMI), 0) + COALESCE((F.PRIMEMI), 0)) AS PRIMEMI_TOTAL
FROM [RR8].[DBO].[RR8_RC_DG] D
LEFT JOIN [RR8].[DBO].[RR8_RC_DG] E
       ON D.CLAVE_COMP = E.CLAVE_COMP AND D.NUMPOL = E.NUMPOL  AND E.ANIO_PARTICION = {año}-1
LEFT JOIN [RR8].[DBO].[RR8_RC_DG] F
       ON D.CLAVE_COMP = F.CLAVE_COMP AND D.NUMPOL = F.NUMPOL  AND F.ANIO_PARTICION = {año}-2
WHERE D.ANIO_PARTICION = {año}
  AND D.INIVIG >= '{fecha_acumulada}'
  AND D.ESTATUS = '01'  ) J
GROUP BY CLAVE_COMP""", myconn)


#Agregar

#Para las pólizas emitidas en el año, por lo menos un registro debe tener una vigencia mayor o igual a un año
pol=pd.read_sql_query(f"""SELECT CLAVE_COMP
, SUM(VIGENCIA) AS ERRORES_VIG_1ANIO
FROM (SELECT CLAVE_COMP,NUMPOL ,max(DIFERENCIA) AS MAXIMMO,
IIF(MAX(DATEDIFF(DAY, INIVIG, FINVIG)) >= 364, 0, 1) AS VIGENCIA
FROM (SELECT CLAVE_COMP,NUMPOL,INIVIG,FINVIG , DATEDIFF(DAY,INIVIG,FINVIG) AS DIFERENCIA
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION = {año} AND YEAR(FECHEMI) = {año})a
GROUP BY CLAVE_COMP,NUMPOL)B
GROUP BY  CLAVE_COMP""",myconn)

#Todos los registro de una misma póliza debe tener el mismo fin de vigencia
reg_pol=pd.read_sql_query(f"""SELECT CLAVE_COMP
                          , SUM(REPITE_VIG) AS ERRORES_POLVIG
FROM(
SELECT CLAVE_COMP, NUMPOL,COUNT(FINVIG) AS FECHAS, IIF(COUNT(FINVIG) > 1, 1, 0) AS REPITE_VIG
FROM (
SELECT DISTINCT CLAVE_COMP, NUMPOL,FINVIG,
FROM [RR8].[dbo].[RR8_RC_DG]
WHERE ANIO_PARTICION = {año}
GROUP BY CLAVE_COMP,NUMPOL,FINVIG
)A
GROUP BY CLAVE_COMP, NUMPOL
)B
GROUP BY CLAVE_COMP""",myconn)


aanteriores=(aant_acum.merge(aant_vigor, on=['CLAVE_COMP'], how='outer')
             .merge(aant_sin, on=['CLAVE_COMP'], how='outer')
             .merge(aant_sinrep, on=['CLAVE_COMP'], how='outer')
             .merge(reg_dgemi, on=['CLAVE_COMP'], how='outer')
             .merge(est,on=['CLAVE_COMP'],how='outer')
             .merge(est_1,on=['CLAVE_COMP'],how='outer')
             .merge(pol,on=['CLAVE_COMP'],how='outer')
             .merge(reg_pol,on=['CLAVE_COMP'],how='outer'))


#--------------------------------------------------------------------------------------------------
#Se realiza la unión de todas las tablas y se utiliza la tabla pivote como base
pivote_dg= pd.merge(left=pivote,right=dg,left_on='CLAVE_COMP', right_on='CLAVE_COMP_DG', how='left')
dg_emi= pd.merge(left=pivote_dg,right=emi,left_on='CLAVE_COMP', right_on='CLAVE_COMP_EMI', how='left')
general= pd.merge(left=dg_emi,right=sin,left_on='CLAVE_COMP', right_on='CLAVE_COMP_SIN', how='left')

df = (general.merge(contables, on=['CLAVE_COMP'], how='left')
        .merge(otros, on=['CLAVE_COMP'], how='left')
        .merge(cons, on=['CLAVE_COMP'], how='left')
        .merge(aanteriores, on=['CLAVE_COMP'], how='left')
        )

df=df.drop(['CLAVE_COMP_EMI', 'CLAVE_COMP_SIN'], axis=1)
df = df.rename(columns={'CLAVE_COMP_DG':'NO_ENTREGA'})
df=df.fillna(0)


#Se exporta la información base
df.to_csv(os.path.join(Directorio_exp,'Ptje_error.csv'),index=False,encoding='latin1')


#Importar tabla
#df=pd.read_csv(os.path.join(Directorio_imp,'Ptje_error.csv'),encoding='latin1')


Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'SQL Server Native Client 11.0' : file not found (0) (SQLDriverConnect)")